<a href="https://colab.research.google.com/github/mar1po5-a/ezen/blob/main/20250518_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain-google-genai
!pip install -q langchain langchain-community chromadb
!pip install -q langchain-huggingface
!pip install -q konlpy

In [ ]:
import os
from google.colab import userdata

In [ ]:
# Gemini API 키 비밀관리 설정 후 진행합니다.
# 코랩 환경에서 좌측 메뉴바의 열쇠 버튼을 누르면 비밀키를 설정할 수 있습니다.
# userdata.get() -> 저장된 비밀 값을 안전하게 불러옴
try:
  os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except userdata.SecretNotFoundError:
  print("ERROR: GOOGLE_API_KEY not found in Colab secrets. Pleas set it.")

In [ ]:
# 제미나이 모델
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
# 벡터 DB
from langchain_community.vectorstores import Chroma
# 문서 표현
from langchain_core.documents import Document
# 텍스트 크기 분할
from langchain_text_splitters import KonlpyTextSplitter
# 프롬프트 생성
from langchain_core.prompts import ChatPromptTemplate
# 체인 내에서 데이터를 그대로 전달
from langchain_core.runnables import RunnablePassthrough
# LLM 출력을 문자열로 변환
from langchain_core.output_parsers import StrOutputParser

import torch
# 허깅 페이스 임베딩 모델
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
# 샘플 데이터
documents_data = [
"""'2025 청년 디지털 커리어 점프업 패키지'
최근 디지털 전환 시대에 발맞춰 청년들의 성공적인 커리어 개발과 안정적인 사회 정착을 지원하기 위해 "2025 청년 디지털 커리어 점프업 패키지"가 새롭게 시행됩니다. 본 정책은 대한민국 국적을 가진 만 19세 이상 만 34세 이하의 미취업 또는 단기 근로 청년이라면 누구나 신청 가능하며, 특히 IT, AI, 빅데이터, 콘텐츠 제작 등 미래 유망 디지털 분야로의 진출을 희망하거나 해당 분야에서 역량을 강화하고자 하는 청년들을 우선적으로 지원합니다. 최종학력이나 전공에 관계없이 지원할 수 있으나, 관련 분야 교육 수료 경험이나 포트폴리오를 제출할 경우 가산점이 부여될 수 있습니다.
"2025 청년 디지털 커리어 점프업 패키지"의 주요 내용은 크게 세 가지로 구성됩니다. 첫째, '디지털 핵심역량 강화 교육 지원금'입니다. 선정된 청년에게는 최대 6개월간 매월 50만원의 교육 훈련비가 지급되어, 정부 인증 교육기관 또는 온라인 플랫폼에서 제공하는 고급 디지털 기술 교육 과정을 수강하는 데 사용할 수 있습니다. 이 지원금은 교육 과정의 종류나 기간에 따라 차등 지급될 수 있으며, 성실히 교육을 이수하고 수료증을 제출하는 청년에게는 추가적으로 50만원의 '역량 달성 축하금'이 일시 지급됩니다.
둘째, '미래설계 커리어 컨설팅 및 네트워킹 지원'입니다. 정책 참여 청년들은 전문 커리어 컨설턴트로부터 1:1 맞춤형 진로 상담, 이력서 및 자기소개서 첨삭, 모의 면접 등의 심층 컨설팅을 최소 3회 이상 제공받게 됩니다. 또한, 현업 전문가 및 동료 청년들과의 네트워킹 기회를 확대하기 위해 정기적인 온·오프라인 밋업(Meet-up) 및 워크숍 참여 비용의 80%를 지원하며, 연간 최대 20만원까지 지원받을 수 있습니다. 이를 통해 청년들은 실질적인 취업 정보를 얻고 인적 네트워크를 구축하는 데 도움을 받을 수 있습니다.
셋째, '디지털 분야 취·창업 초기 정착 지원금'입니다. 디지털 핵심역량 강화 교육을 성공적으로 수료하고, 해당 분야로 6개월 이내에 취업하거나 창업에 성공한 청년에게는 초기 정착을 돕기 위한 지원금이 지급됩니다. 취업자의 경우, 정규직으로 3개월 이상 근속 시 월 30만원씩 최대 3개월간, 총 90만원의 '취업 성공 장려금'을 받을 수 있습니다. 창업자의 경우에는 사업자 등록 후 초기 운영자금 명목으로 150만원을 일시금으로 지원하며, 이후 1년간 세무 및 법률 자문 서비스를 무료로 제공받을 수 있는 바우처도 함께 제공됩니다. 단, 이 지원금은 다른 정부 및 지자체의 유사 사업 지원금과 중복 수혜가 제한될 수 있으니 신청 시 유의해야 합니다.
본 정책의 신청 기간은 2025년 3월 1일부터 3월 31일까지이며, 공식 정책 안내 웹사이트를 통해 온라인으로만 접수 가능합니다. 제출 서류로는 신분증 사본, 최종학력 증명서, 구직활동 증명서(또는 단기 근로 계약서), 그리고 선택적으로 디지털 분야 관련 교육 수료증 및 포트폴리오 등이 요구됩니다. 선정 결과는 서류 심사 및 필요한 경우 비대면 인터뷰를 거쳐 5월 초에 개별 통보될 예정입니다. "2025 청년 디지털 커리어 점프업 패키지"를 통해 많은 청년들이 디지털 시대의 주역으로 성장할 수 있기를 기대합니다.""",

"""2025 청년 주거안정 스텝업 프로그램
2025년부터 청년층의 주거 문제 해결과 자산형성을 지원하기 위한 "청년 주거안정 스텝업 프로그램"이 전국적으로 확대 시행됩니다. 본 정책은 만 19세 이상 34세 이하의 무주택 청년을 대상으로 하며, 특히 월평균 소득이 중위소득 120% 이하인 청년들에게 우선 지원 혜택이 제공됩니다. 1인 가구와 신혼부부(혼인 7년 이내)는 소득 기준을 완화하여 중위소득 140%까지 지원 가능합니다.
"청년 주거안정 스텝업 프로그램"의 핵심 지원 내용은 크게 세 가지로 구성됩니다. 첫째, '월세 지원 바우처'입니다. 선정된 청년에게는 지역별 임대료 수준에 따라 차등적으로 매월 10만원에서 30만원까지의 월세 지원금이 최대 2년간 지급됩니다. 서울 및 수도권은 월 30만원, 광역시는 월 20만원, 그 외 지역은 월 10만원이 기본 지원되며, 주택 크기와 월세 부담률에 따라 추가 지원이 가능합니다.
둘째, '청년 주택 구입 저금리 대출' 제도입니다. 만 25세 이상 34세 이하의 청년이 생애 첫 주택을 구입할 경우, 최대 3억원까지 연 1.5% 고정금리로 대출을 제공합니다. 대출 기간은 최대 30년이며, 첫 5년간은 이자만 납부하는 거치기간 선택도 가능합니다. 또한 대출 실행 후 3년 이내에 결혼하거나 출산할 경우, 대출 금리가 0.3%p 추가 인하되는 혜택도 제공됩니다.
셋째, '청년 주거환경 개선 지원금'입니다. 낙후된 주택에 거주하는 청년들에게 주거 품질 향상을 위한 리모델링 비용을 최대 500만원까지 지원합니다. 에너지 효율화(단열재 교체, LED 조명 설치 등), 안전시설 설치(화재경보기, 가스누출 감지기 등), 그리고 기본 생활 편의시설(욕실, 주방 개선 등) 리모델링이 지원 대상입니다. 단, 임차인의 경우 집주인의 동의서가 필요하며, 지원금 사용 후 최소 2년간 해당 주택에 거주해야 한다는 조건이 있습니다.
본 정책의 신청 기간은 2025년 2월 1일부터 2월 28일까지이며, 각 지자체 주택정책과 또는 LH공사 청년주택지원센터를 통해 접수 가능합니다. 제출 서류로는 신분증, 주민등록등본, 소득금액증명원(또는 근로소득원천징수영수증), 임대차계약서(임차인의 경우), 그리고 주택 상태 사진(주거환경 개선 지원 신청 시)이 필요합니다. 선정 결과는 3월 중순에 개별 통보되며, 지원금은 4월부터 순차적으로 지급될 예정입니다.
"청년 주거안정 스텝업 프로그램"을 통해 많은 청년들이 안정적인 주거환경에서 미래를 준비하고, 궁극적으로는 내 집 마련의 꿈을 이룰 수 있기를 기대합니다.""",

 """2025 청년 글로벌 성장 드림 프로젝트
급변하는 글로벌 환경 속에서 청년들의 국제 경쟁력 강화와 다양한 글로벌 경험 기회 확대를 위해 "2025 청년 글로벌 성장 드림 프로젝트"가 새롭게 출범합니다. 본 프로젝트는 만 19세 이상 만 35세 이하의 대한민국 국적 청년이면 누구나 지원 가능하며, 특히 해외 진출 의지가 강한 취업준비생, 예비 창업가, 그리고 대학(원)생을 우선적으로 선발합니다. 영어 및 제2외국어 능력은 선발 과정에서 참고자료로 활용되나, 선발의 절대적 기준은 아니며 지원자의 열정과 명확한 목표 의식이 더 중요한 평가 요소로 작용합니다.
"2025 청년 글로벌 성장 드림 프로젝트"는 크게 네 가지 핵심 프로그램으로 구성됩니다. 첫째, '글로벌 인턴십 및 연수 지원'입니다. 매년 200명의 청년들에게 해외 유수 기업 및 국제기구에서 3~6개월간 인턴십을 수행할 수 있는 기회가 제공됩니다. 선발된 청년에게는 왕복 항공권, 월 최대 150만원의 체재비, 그리고 해외 의료보험 가입비가 지원됩니다. 또한 현지 적응 및 업무 수행을 위한 사전 교육 프로그램(온라인 어학 강좌, 글로벌 비즈니스 매너 등)도 무료로 제공됩니다.
둘째, '청년 글로벌 창업 지원사업'입니다. 해외 시장을 겨냥한 창의적인 아이디어와 비즈니스 모델을 보유한 청년 (예비)창업가 50팀을 선발하여 팀당 최대 5,000만원의 초기 사업화 자금을 지원합니다. 더불어 해외 시장 조사비, 글로벌 액셀러레이터 프로그램 참가비, 해외 전시회 및 박람회 참가비도 별도로 지원되며, 글로벌 멘토링 시스템을 통해 현지 시장에 정통한 전문가로부터 1:1 컨설팅을 받을 수 있습니다.
셋째, '글로벌 역량 강화 교육 바우처'입니다. 선정된 청년 1,000명에게 연간 최대 300만원의 교육 바우처가 지급되어, 외국어 학습, 국제 자격증 취득, 글로벌 비즈니스 스킬 향상 등을 위한 다양한 교육 프로그램에 참여할 수 있습니다. 온라인 강좌, 오프라인 학원, 1:1 튜터링 등 교육 형태에 제한 없이 자유롭게 선택 가능하며, 분기별로 교육 성과 보고서를 제출하면 다음 분기 바우처가 지급되는 방식으로 운영됩니다.
넷째, '글로벌 네트워킹 및 문화교류 프로그램'입니다. 분기별로 개최되는 국제 청년 포럼, 글로벌 해커톤, 문화교류 페스티벌 등에 참가할 수 있는 기회가 제공됩니다. 이러한 행사를 통해 세계 각국의 청년들과 교류하며 글로벌 네트워크를 구축하고 다양한 문화에 대한 이해도를 높일 수 있습니다. 해외 개최 행사의 경우, 참가비의 70%와 왕복 항공권이 지원되며, 국내 개최 행사는 전액 무료로 참가할 수 있습니다.
본 프로젝트의 신청 기간은 2025년 4월 1일부터 4월 30일까지이며, 공식 웹사이트(www.globaldream2025.go.kr)를 통해 온라인으로 접수 가능합니다. 제출 서류로는 신분증 사본, 영문 이력서, 지원 동기 및 계획서(영문 또는 한글), 어학능력 증빙자료(선택사항), 포트폴리오 또는 관련 활동 증빙자료(선택사항)가 요구됩니다. 서류심사와 화상면접을 거쳐 최종 선발 결과는 6월 중순에 발표될 예정입니다.
"2025 청년 글로벌 성장 드림 프로젝트"를 통해 대한민국 청년들이 세계 무대에서 자신의 꿈을 펼치고 글로벌 인재로 성장하는 데 큰 도움이 되기를 기대합니다.""",

"""2025 청년 창의혁신 라이즈업 이니셔티브
급변하는 디지털 시대에 청년들의 창의적 역량과 혁신 정신을 발굴하고 육성하기 위한 "2025 청년 창의혁신 라이즈업 이니셔티브"가 전국적으로 시행됩니다. 본 정책은 만 18세 이상 만 39세 이하의 창의적인 아이디어와 혁신적인 프로젝트를 추진하고자 하는 모든 청년들을 대상으로 합니다. 특히 문화예술, 과학기술, 사회혁신, 친환경 분야 등 미래 성장동력 영역에서 새로운 가치를 창출하고자 하는 청년들에게 포괄적인 지원을 제공합니다.
"2025 청년 창의혁신 라이즈업 이니셔티브"는 크게 네 가지 지원 트랙으로 구성됩니다. 첫째, '창의 프로젝트 씨드머니 지원'입니다. 혁신적인 아이디어를 가진 청년 개인 또는 팀에게 프로젝트당 최대 2,000만원의 초기 자금을 지원합니다. 지원 분야는 △디지털 콘텐츠 △신재생에너지 △바이오헬스 △스마트시티 △메타버스 △친환경 솔루션 등 미래 유망 분야를 중심으로 하되, 분야 제한 없이 창의성과 실현 가능성이 뛰어난 프로젝트라면 모두 지원 대상이 됩니다. 선정된 프로젝트는 6개월간의 개발 기간 동안 중간 평가를 거쳐 추가 자금도 지원받을 수 있습니다.
둘째, '창의혁신 공간 및 장비 지원'입니다. 전국 주요 도시에 설치된 '청년 창의혁신 허브'를 무료로 이용할 수 있는 권한이 주어집니다. 이 공간은 24시간 개방되며, 고성능 컴퓨터, 3D 프린터, 영상 제작 장비, VR/AR 기기 등 최신 창작 도구와 장비를 갖추고 있습니다. 또한 필요에 따라 전문 장비를 대여할 수 있는 월 최대 50만원의 장비 대여 바우처도 제공됩니다. 공간 내에서는 정기적으로 전문가 특강, 네트워킹 이벤트, 아이디어 공유회 등이 개최되어 다양한 분야의 청년들이 교류하며 시너지를 창출할 수 있는 기회를 제공합니다.
셋째, '혁신 역량 강화 교육 프로그램'입니다. 창의적 문제 해결 능력, 디자인 씽킹, 기술 트렌드, 지적재산권 관리, 비즈니스 모델링 등 혁신에 필요한 다양한 역량을 강화할 수 있는 온·오프라인 교육과정이 제공됩니다. 선정된 청년들은 연간 최대 10개 과정을 무료로 수강할 수 있으며, 수료 시 공인 인증서가 발급됩니다. 또한 분야별 최고 전문가들과의 1:1 멘토링 기회가 연 5회 제공되어, 프로젝트 발전 방향에 대한 맞춤형 조언을 받을 수 있습니다.
넷째, '창의혁신 성과 확산 및 사업화 지원'입니다. 프로젝트가 완성 단계에 이르면, 투자자, 기업, 정부기관 등이 참여하는 '청년 혁신 쇼케이스'에 참가할 수 있는 기회가 제공됩니다. 또한 우수 프로젝트로 선정될 경우, 추가로 최대 5,000만원의 사업화 자금을 지원받을 수 있으며, 특허 출원 및 등록 비용 전액, 국내외 전시회 참가비, 마케팅 비용 등도 지원됩니다. 특히 사회적 가치가 큰 프로젝트의 경우, 정부 및 공공기관과의 시범사업 기회도 제공됩니다.
본 정책의 신청 기간은 2025년 5월 1일부터 5월 31일까지이며, 공식 웹사이트(www.youthriseupi.kr)를 통해 온라인으로 접수 가능합니다. 제출 서류로는 신분증 사본, 프로젝트 계획서(아이디어 소개, 추진 방법, 예상 결과물, 필요 자원 등 포함), 포트폴리오(선택사항), 그리고 2분 이내의 영상 피치(선택사항)가 요구됩니다. 서류심사와 발표심사를 거쳐 7월 중순에 최종 선정 결과가 발표될 예정입니다.
"2025 청년 창의혁신 라이즈업 이니셔티브"를 통해 대한민국 청년들의 창의적 아이디어가 현실이 되고, 이를 통해 개인의 성장뿐만 아니라 사회 혁신에도 기여할 수 있기를 기대합니다.""",

"""2025 청년 금융 자립 레벨업 플랜
불안정한 고용환경과 자산형성의 어려움에 직면한 청년층의 경제적 자립과 건전한 금융 생활을 지원하기 위해 "2025 청년 금융 자립 레벨업 플랜"이 시행됩니다. 본 정책은 만 19세 이상 만 34세 이하의 근로 소득이 있는 청년 또는 창업 준비 중인 청년을 대상으로 하며, 특히 중위소득 150% 이하 청년들에게 우선적인 혜택이 제공됩니다. 학력이나 직업에 관계없이 신청 가능하며, 금융 취약계층(신용점수 하위 30%, 금융교육 이수 경험 없는 자 등)에게는 가산점이 부여됩니다.
"2025 청년 금융 자립 레벨업 플랜"은 크게 네 가지 핵심 프로그램으로 구성됩니다. 첫째, '청년 자산형성 매칭 저축'입니다. 매월 10만원에서 50만원까지 정기적으로 저축하는 청년에게 정부가 동일한 금액을 매칭하여 최대 3년간 지원합니다. 매칭 비율은 소득 수준에 따라 차등 적용되어, 중위소득 100% 이하는 1:1, 100130%는 1:0.8, 130150%는 1:0.5의 비율로 지원받을 수 있습니다. 이 자금은 주택 구입, 임차보증금, 교육비, 창업자금, 결혼자금 등 미래 설계를 위한 용도로만 사용 가능하며, 만기 시 목적 외 사용 시에는 정부 지원금의 이자만 지급됩니다.
둘째, '청년 부채 관리 및 신용 회복 프로그램'입니다. 학자금 대출, 생계형 대출 등으로 재정적 어려움을 겪는 청년들에게 맞춤형 부채 관리 솔루션을 제공합니다. 구체적으로 △부채 진단 및 맞춤형 상환 계획 수립 △최대 3년간 금리 인하(최대 3%p) △상환 유예 기간 연장(최대 2년) △분할 상환 기간 확대(최대 10년) 등의 혜택을 받을 수 있습니다. 또한 성실 상환자에게는 매년 원금의 5%(최대 100만원)를 인센티브로 지원하며, 신용 회복 중인 청년에게는 전문 신용 컨설팅과 함께 소액 신용 대출 기회(연 3% 저금리, 최대 500만원)도 제공됩니다.
셋째, '청년 금융 리터러시 향상 프로그램'입니다. 체계적인 금융교육을 통해 청년들의 금융 이해력과 관리 능력을 향상시키는 것을 목표로 합니다. 온라인 플랫폼을 통해 △기초 금융 상식 △투자 원리 △세금 및 연금 이해하기 △부동산 경제 △보험 활용법 등 생애주기별 필수 금융 지식을 학습할 수 있으며, 전 과정 이수 시 100만원의 '금융교육 이수 장려금'이 지급됩니다. 더불어 오프라인 '청년 금융 캠프'가 분기별로 개최되어, 실전 자산관리 시뮬레이션, 투자 클리닉, 세무 상담 등 실용적인 교육과 네트워킹 기회를 제공합니다.
넷째, '청년 창업자 금융 지원'입니다. 창업을 준비하거나 창업 초기 단계에 있는 청년들에게 특화된 금융 지원을 제공합니다. 구체적으로 △최대 5,000만원의 저금리(연 1%) 창업 대출 △창업 후 2년간 원금 상환 유예 △분기별 세무 및 회계 컨설팅 무료 제공 △사업자 신용평가 관리 지원 등의 혜택이 포함됩니다. 또한 성공적으로 사업을 운영하며 일자리를 창출하는 청년 창업자에게는 대출 원금의 최대 20%를 감면해주는 '일자리 창출 인센티브'도 마련되어 있습니다.
본 정책의 신청 기간은 2025년 3월 15일부터 4월 15일까지이며, 각 지자체 청년지원센터 또는 금융기관(국민/신한/우리/하나/농협 은행)을 통해 접수 가능합니다. 제출 서류로는 신분증, 주민등록등본, 소득금액증명원(또는 근로소득원천징수영수증), 금융거래 내역서(최근 1년), 그리고 향후 자금 활용 계획서가 필요합니다. 서류심사를 거쳐 5월 중순에 선정 결과가 발표되며, 선정된 청년들은 의무적으로 기초 금융교육(8시간)을 이수한 후 6월부터 본격적인 프로그램 혜택을 받게 됩니다.
"2025 청년 금융 자립 레벨업 플랜"을 통해 많은 청년들이 경제적 자립 기반을 마련하고, 건강한 금융 습관을 형성하여 보다 안정적인 미래를 설계할 수 있기를 기대합니다."""
]

In [ ]:
# 샘플 데이터를 Document 객체로 변환
# page_context와 metadata를 가집니다.
docs = [Document(page_content=text, metadata={"source": f"doc_{i+1}"})
for i, text in enumerate(documents_data)]

print(docs)

In [ ]:
# 검색 정확도를 높이고, 토큰을 효율적으로 활용하기 위해 chunk 단위로 분할
text_splitter = KonlpyTextSplitter(
    chunk_size = 700, # 각 청크의 크기
    chunk_overlap = 70 # 청크 간 중복되는 글자 수
)
# split_documents -> 그냥 변수명
# .split_documents() -> method
split_documents = text_splitter.split_documents(docs)

print(f"원본 문서 개수: {len(docs)}")
print(f"분할된 청크 개수: {len(split_documents)}")
# print("\n분할된 첫 번째 청크 예시:")
print(split_documents[7])

In [ ]:
# 임베딩 모델 초기화
# KURE - 고려대학교 한국어 기반 임베딩 모델
model_name_or_path = "nlpai-lab/KURE-v1"

# --- LangChain의 HuggingFaceEmbeddings 래퍼 사용 ---
# 허깅페이스 HF_TOKEN 비밀 키 설정이 필요합니다. 회원 가입 후 READ 토큰 생성
# HuggingFaceEmbeddings 클래스가 내부적으로 모델 로드 및 임베딩 추출을 처리해줍니다.
try:
    # model_name에 Hugging Face Hub의 모델 ID 또는 로컬 경로를 지정합니다.
    # model_kwargs는 모델 로드 시 전달할 추가 인자입니다 (예: device 설정).
    # encode_kwargs는 텍스트를 인코딩(임베딩)할 때 전달할 인자입니다 (예: normalize_embeddings).
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name_or_path,
        model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}, # GPU 사용 가능하면 GPU, 아니면 CPU
        encode_kwargs={'normalize_embeddings': True} # 임베딩 정규화 (선택 사항, 성능에 영향 줄 수 있음)
    )
    # 간단한 테스트
    test_query_embedding = embeddings.embed_query("샘플 데이터")
    print(f"한국어 임베딩 모델 ('{model_name_or_path}')이 HuggingFaceEmbeddings를 통해 성공적으로 로드 및 테스트되었습니다.")
    print(f"테스트 임베딩 차원: {len(test_query_embedding)}")

except Exception as e:
    print(f"HuggingFaceEmbeddings 초기화 중 오류 발생: {e}")
    print("모델 이름이나 경로가 올바른지, 필요한 라이브러리(transformers, sentence-transformers 등)가 설치되었는지 확인해주세요.")
    raise

# llm 초기화
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-preview-04-17",
    temperature=0.3 # 창의성
    # 시스템 프롬프트를 사용자 프롬프트 형태로 변환하여 전달
    # API 호환성과 model의 특성에 따라 유동적으로 선택하는 옵션
    # convert_system_message_to_human=True
)

print("KURE 임베딩 모델 및 Gemini LLM이 성공적으로 초기화되었습니다.")

In [ ]:
# 벡터 저장소 생성
# Chorma.from_documents()는 모든 작업을 한 번에 수행
# 1. 문서 청크를 임베딩 모델을 사용해 벡터로 변환
# 2. 벡터와 원본 텍스트, 메타 데이터를 ChromaDB에 저장
# persist_directory를 지정하면 해당 경로에 DB 파일이 저장됨. -> 재사용 가능
vectorstore_path = "./chroma_db_rag_store"

vectorstore = Chroma.from_documents(
    documents=split_documents, # 분할된 문서
    embedding=embeddings, # 임베딩 모델
    # persist : 지속하다
    # persist_directory=vectorstore_path # DB 저장할 디렉토리
)

# 저장된 DB를 다시 로드할 때 사용 (재생성 x)
# vectorstore = Chroma(persist_directory=vectorstore_path, embedding_function=embeddings)

print(f"문서들이 ChromaDB에 성공적으로 임베딩 및 저장되었습니다. (저장 경로: {vectorstore_path})")
# 기존의 RDB : table / Vector DB : collection
print(f"ChromaDB에 저장된 총 벡터(청크) 수: {vectorstore._collection.count()}")

In [ ]:
# 벡터 저장소에서 유사도 높은 문서를 검색하는 검색기 생성
# retriever : 되찾다, 회수하다
# k=3은 가장 유사한 문서 3개를 가져오도록 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 검색기 테스트
query_test = "점프업 패키지의 지원 조건은 어떻게 되나요? 제가 32세인데 참여 가능할까요?"
retrieved_docs_test = retriever.invoke(query_test)
print(f"'{query_test}'에 대한 검색 결과 (상위 {len(retrieved_docs_test)}개)")
for i, doc in enumerate(retrieved_docs_test):
  # 메타데이터 중 source를 가져오되, 존재하지 않으면 기본 값을 가져옴
  print(f" {i+1}, {doc.page_content} (출처: {doc.metadata.get('source', 'N/A')})")

In [ ]:
# RAG 파이프라인 구성 : 정보 검색 - 문맥 정보 준비 - 프롬프트 구성
# - 답변 생성 및 출력 일련의 과정이 파이프라인

# 프롬프트 템플릿 정의
# {context} : 검색된 문서 내용, {question} : 사용자 질
prompt_template = """당신은 대한민국 정부에서 주관하는 청년 정책의 정확한 정보를 제공하는 AI 어시스턴트입니다.
주어진 문맥(context) 정보를 바탕으로 질문에 답변해주세요. 문맥에서 답을 찾을 수 없다면, "제공된 정보만으로는 답변하기 어렵습니다."라고 솔직하게 말해주세요.

문맥:
{context}

질문:
{question}

답변:
"""

prompt = ChatPromptTemplate.from_template(prompt_template)

# RAG 체인 정의
# retriever로 문서를 검색하고
# 검색된 문서들의 page_content를 하나의 문자열로 합
# LCEL 사용 : |(파이프 기호)의 사용, 파이프라인 또는 체인을 더 쉽고 유연하게 구성하는 방식
rag_chain = (
    {"context": retriever | (lambda docs: "\n\n".join(d.page_content for d in docs)),
    # RunnablePassthrough.assian() : 체인 내에서 새로운 키를 생성하고 값을 할당
     # question에 초기 입력을 그대로 할당하기 위해 키:값으로 세팅
     "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser() # llm 출력을 일반 문자열로 변환
)

print("RAG 체인이 성공적으로 구성되었습니다.")


In [ ]:
questions_list = [
"점프업 패키지의 지원 조건은 어떻게 되나요? 제가 32세인데 참여 가능할까요?",
"청년 주거안정 스텝업 프로그램에서 월세 지원 바우처는 어떤 방식으로 받을 수 있나요? 지원금이 집주인에게 직접 지급되나요, 아니면 제 계좌로 입금되나요?",
"글로벌 성장 드림 프로젝트에 지원하려면 영어 성적이 필수인가요? TOEIC이나 TOEFL 점수가 없어도 지원할 수 있을까요?",
"창의혁신 라이즈업 이니셔티브에서 지원하는 분야가 궁금합니다. 제가 문화예술 분야 프로젝트를 계획 중인데 지원 대상이 될 수 있나요?",
"청년 금융 자립 레벨업 플랜의 자산형성 매칭 저축에 가입하면 정확히 얼마를 지원받을 수 있나요? 제 월급이 250만원인데 매칭 비율이 어떻게 되나요?",
"점프업 패키지의 교육 지원금으로 해외 온라인 교육 프로그램을 수강해도 지원금을 받을 수 있나요?",
"청년 주거안정 프로그램의 주택 구입 대출을 받으려면 어떤 서류가 필요한가요? 현재 대출이 있어도 신청 가능한가요?",
"창의혁신 라이즈업 이니셔티브와 글로벌 성장 드림 프로젝트를 동시에 지원해도 되나요? 두 정책 모두에서 지원금을 받을 수 있을까요?",

# 샘플 데이터에 명시되지 않은 부분에 대한 질문
"점프업 패키지 수혜자가 교육 중간에 취업이 된 경우에도 계속해서 지원금을 받을 수 있나요? 취업 후 교육을 계속 진행해도 될까요?",
"청년 금융 자립 레벨업 플랜에서 제공하는 금융 교육을 이수한 후에 자격증 시험 준비를 위한 추가 지원이 있나요? 특히 AFPK나 CFP 같은 금융 자격증 취득을 목표로 하고 있습니다.",
"글로벌 성장 드림 프로젝트에 선발된 후 코로나와 같은 팬데믹이나 국제 정세 불안으로 해외 파견이 어려워질 경우 대체 프로그램이 있나요? 온라인으로 전환되거나 국내 활동으로 대체될 수 있는지 궁금합니다."
]

# 각 질문에 대해 RAG 체인을 실행하고 답변을 출력합니다.
for i, question_text in enumerate(questions_list):
    # 질문 번호는 실제 리스트 인덱스에 맞춰서 출력 (또는 원하는 시작 번호로 조정)
    print(f"\n[질문 {i + 1}]: {question_text}") # 여기서는 리스트 순서대로 1번부터 출력
    # 만약 6번부터 번호를 매기고 싶다면 print(f"\n[질문 {i + 6}]: {question_text}") 와 같이 조정
    answer = rag_chain.invoke(question_text)
    print(f"[답변 {i + 1}]: {answer}")

In [ ]:
# 해당 코드는 임베딩 모델의 성능을 확인하기 위해 작성된 코드입니다.
# 사용자의 질문, 임베딩 모델이 검색한 문서, LLM의 답변을 함께 출력하고 있습니다.

# 각 질문에 대해 RAG 체인을 실행하고 답변을 출력합니다.
for i, question_text in enumerate(questions_list):
    question_number = i + 1 # 실제 질문 번호 (1부터 시작)
    # 만약 6번부터 번호를 매기고 싶다면 question_number = i + 6

    print(f"\n\n--- [질문 {question_number}: {question_text}] ---")

    # 1. 해당 질문에 대한 문맥(검색된 문서) 확인
    # Retriever를 직접 호출하여 검색 결과를 가져옵니다.
    # rag_chain 내부의 retriever와 동일한 retriever 객체를 사용해야 합니다.
    # (이전 셀에서 'retriever'라는 이름으로 생성했다고 가정)
    try:
        retrieved_docs = retriever.invoke(question_text) # retriever 직접 호출
        print(f"\n[검색된 문맥 정보 (상위 {len(retrieved_docs)}개)]:")
        if not retrieved_docs:
            print("  -> 검색된 문서가 없습니다.")
        for j, doc in enumerate(retrieved_docs):
            print(f"  [문맥 {j+1}] 출처: {doc.metadata.get('source', 'N/A')}")
            print(f"  내용: {doc.page_content}\n")

        # 검색된 문서 내용을 하나의 문자열로 합치기 (RAG 체인 내부 로직과 유사하게)
        context_for_llm = "\n\n".join(d.page_content for d in retrieved_docs)
        # print(f"\n[LLM에 전달될 최종 문맥]:\n{context_for_llm}") # 너무 길면 주석 처리

    except Exception as e:
        print(f"  -> 문맥 검색 중 오류 발생: {e}")
        context_for_llm = "문맥 정보를 가져오는 데 실패했습니다." # 오류 시 기본 문맥

    # 2. RAG 체인 실행하여 답변 생성
    # 이제 rag_chain은 내부적으로 retriever를 다시 호출하겠지만,
    # 우리는 이미 어떤 문맥이 사용될지 확인했습니다.
    # 만약 retriever 호출을 한 번만 하고 싶다면, rag_chain 구성을 약간 수정해야 합니다.
    # (예: retriever 결과를 직접 rag_chain의 context 입력으로 전달)
    # 여기서는 단순 확인을 위해 기존 rag_chain을 그대로 사용합니다.
    print(f"\n[LLM 답변 생성 중...]")
    try:
        answer = rag_chain.invoke(question_text)
        print(f"[답변 {question_number}]: {answer}")
    except Exception as e:
        print(f"  -> LLM 답변 생성 중 오류 발생: {e}")
        print(f"[답변 {question_number}]: 답변을 생성하는 데 실패했습니다.")